# DeepLearning

## MNIST Dataset
The MNIST database is a well-known academic dataset used to benchmark
classification performance. The data consists of 60,000 training images and
10,000 test images. Each image is a standardized $28^2$ pixel greyscale image of
a single handwritten digit. An example of the scanned handwritten digits is
shown
![Example MNIST digit images](../../h2o-docs/src/booklets/v2_2015/source/images/mnistdigits.jpg)

In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,6 mins 46 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,21 hours
H2O cluster name:,fmilo
H2O cluster total nodes:,1
H2O cluster free memory:,12.44 Gb
H2O cluster total cores:,40
H2O cluster allowed cores:,40
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [2]:
test_df = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/mnist/test.csv.gz")

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


In [3]:
train_df = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/mnist/train.csv.gz")

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


In [4]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

Specify the response and predictor columns

In [5]:
y = "C785"
x = train_df.names[0:784]

In [6]:
train_df[y] = train_df[y].asfactor()
test_df[y] = test_df[y].asfactor()

Train Deep Learning model and validate on test set

In [7]:
model = H2ODeepLearningEstimator(
   distribution="multinomial",
   activation="RectifierWithDropout",
   hidden=[32, 32, 32],
   input_dropout_ratio=0.2,
   sparse=True,
   l1=1e-5,
   epochs=10)

In [8]:
model.train(
    x=x, 
    y=y,
    training_frame=train_df,
    validation_frame=test_df
)

deeplearning Model Build progress: |████████████████████████████████████████████████████████████████| 100%


## Using Crossvalidation

If the value specified for nfolds is a positive integer, N-fold cross-validation is
performed on the training frame and the cross-validation metrics are computed
and stored as model output. 

To disable cross-validation, use `nfolds=0`, which is the default value.

Advanced users can also specify a fold column that defines the holdout
fold associated with each row. By default, the holdout fold assignment is
random. H2O supports other schemes such as round-robin assignment using the modulo
operator.

Perform 5-fold cross-validation on training_frame

In [9]:
model_crossvalidated = H2ODeepLearningEstimator(
    distribution="multinomial",
    activation="RectifierWithDropout",
    hidden=[32,32,32],
    input_dropout_ratio=0.2,
    sparse=True,
    l1=1e-5,
    epochs=10,
    nfolds=5
)

In [10]:
model_crossvalidated.train(
    x=x,
    y=y,
    training_frame=train_df
)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


## Extracting and Handling the Results

We can now extract the parameters of our model, examine the scoring process,
and make predictions on new data.

In [11]:
# View specified parameters of the Deep Learning model
model.params;

In [12]:
# Examine the performance of the trained model
model # display all performance metrics

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1476206324318_1
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 25,418 weights/biases, 406.4 KB, 600,000 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,20.0,,,,,,,,,
,2,32,RectifierDropout,50.0,1e-05,0.0,0.0322727,0.1822508,0.0,-0.0091462,0.0640087,0.5111540,0.2292953
,3,32,RectifierDropout,50.0,1e-05,0.0,0.0003422,0.0002376,0.0,-0.0280022,0.1985070,0.7924165,0.3739460
,4,32,RectifierDropout,50.0,1e-05,0.0,0.0005674,0.0002879,0.0,-0.0491354,0.2136040,0.7378433,0.4151546
,5,10,Softmax,,1e-05,0.0,0.0027797,0.0024106,0.0,-0.4225928,1.0700207,-1.9734173,0.9454455




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.11735399631
RMSE: 0.342569695551
LogLoss: 0.401746649673
Mean Per-Class Error: 0.0906898376913
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
919.0,0.0,15.0,12.0,0.0,66.0,2.0,0.0,5.0,1.0,0.0990196,"101 / 1,020"
0.0,1052.0,14.0,2.0,2.0,7.0,0.0,5.0,26.0,2.0,0.0522523,"58 / 1,110"
1.0,1.0,986.0,23.0,9.0,5.0,1.0,13.0,13.0,1.0,0.0636277,"67 / 1,053"
0.0,2.0,34.0,988.0,0.0,21.0,0.0,7.0,8.0,5.0,0.0723005,"77 / 1,065"
0.0,0.0,12.0,1.0,838.0,7.0,1.0,3.0,5.0,69.0,0.1047009,98 / 936
3.0,0.0,15.0,62.0,6.0,723.0,3.0,2.0,27.0,4.0,0.1443787,122 / 845
3.0,0.0,38.0,0.0,7.0,31.0,872.0,0.0,8.0,0.0,0.0907195,87 / 959
3.0,2.0,7.0,7.0,4.0,2.0,0.0,964.0,0.0,45.0,0.0676983,"70 / 1,034"
2.0,13.0,16.0,19.0,6.0,45.0,2.0,2.0,907.0,12.0,0.1142578,"117 / 1,024"
3.0,2.0,0.0,18.0,24.0,17.0,1.0,26.0,9.0,921.0,0.0979432,"100 / 1,021"


Top-10 Hit Ratios: 


...

0,1,2,3,4,5,6,7,8,9,Error,Rate
898.0,0.0,2.0,11.0,0.0,60.0,5.0,3.0,1.0,0.0,0.0836735,82 / 980
0.0,1096.0,9.0,6.0,0.0,0.0,0.0,1.0,22.0,1.0,0.0343612,"39 / 1,135"
1.0,4.0,955.0,27.0,9.0,6.0,3.0,9.0,17.0,1.0,0.0746124,"77 / 1,032"
0.0,0.0,22.0,939.0,0.0,23.0,0.0,10.0,12.0,4.0,0.0702970,"71 / 1,010"
0.0,0.0,14.0,0.0,892.0,8.0,3.0,2.0,5.0,58.0,0.0916497,90 / 982
3.0,1.0,13.0,75.0,5.0,749.0,1.0,1.0,32.0,12.0,0.1603139,143 / 892
7.0,3.0,27.0,2.0,11.0,46.0,854.0,1.0,7.0,0.0,0.1085595,104 / 958
1.0,6.0,28.0,13.0,2.0,1.0,0.0,928.0,0.0,49.0,0.0972763,"100 / 1,028"
0.0,6.0,13.0,17.0,10.0,46.0,4.0,8.0,851.0,19.0,0.1262834,123 / 974
0.0,3.0,2.0,13.0,29.0,21.0,0.0,17.0,14.0,910.0,0.0981169,"99 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9072
2,0.9545000
3,0.9715999
4,0.9803000
5,0.9880999
6,0.9933999
7,0.9965999
8,0.9978999
9,0.9989999
10,0.9999999


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-11 10:25:48,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-11 10:25:49,4.996 sec,67189 obs/sec,1.0,1,60000.0,0.5180204,0.7838807,0.2520115,0.5113724,0.7644453,0.2449
,2016-10-11 10:25:55,11.074 sec,97213 obs/sec,10.0,10,600000.0,0.3425697,0.4017466,0.0891030,0.3404794,0.4027760,0.0928


In [13]:
model.model_performance(train=True) # training metrics
model.model_performance(valid=True) # validation metrics


ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.115926192101
RMSE: 0.340479356351
LogLoss: 0.402776017504
Mean Per-Class Error: 0.0945143810095
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
898.0,0.0,2.0,11.0,0.0,60.0,5.0,3.0,1.0,0.0,0.0836735,82 / 980
0.0,1096.0,9.0,6.0,0.0,0.0,0.0,1.0,22.0,1.0,0.0343612,"39 / 1,135"
1.0,4.0,955.0,27.0,9.0,6.0,3.0,9.0,17.0,1.0,0.0746124,"77 / 1,032"
0.0,0.0,22.0,939.0,0.0,23.0,0.0,10.0,12.0,4.0,0.0702970,"71 / 1,010"
0.0,0.0,14.0,0.0,892.0,8.0,3.0,2.0,5.0,58.0,0.0916497,90 / 982
3.0,1.0,13.0,75.0,5.0,749.0,1.0,1.0,32.0,12.0,0.1603139,143 / 892
7.0,3.0,27.0,2.0,11.0,46.0,854.0,1.0,7.0,0.0,0.1085595,104 / 958
1.0,6.0,28.0,13.0,2.0,1.0,0.0,928.0,0.0,49.0,0.0972763,"100 / 1,028"
0.0,6.0,13.0,17.0,10.0,46.0,4.0,8.0,851.0,19.0,0.1262834,123 / 974
0.0,3.0,2.0,13.0,29.0,21.0,0.0,17.0,14.0,910.0,0.0981169,"99 / 1,009"


Top-10 Hit Ratios: 


k,hit_ratio
1,0.9072
2,0.9545000
3,0.9715999
4,0.9803000
5,0.9880999
6,0.9933999
7,0.9965999
8,0.9978999
9,0.9989999
10,0.9999999


The validation error is based on the
parameter `score validation samples`, which configures the same value
on the validation set (by default, this is the entire validation set).

In [14]:
# Get MSE only
model.mse(valid=True)

0.11592619210106618

In [15]:
# Cross-validated MSE
model_crossvalidated.mse(xval=True)

0.10834941601018695

In [16]:
ls ../../h2o-docs/src/booklets/v2_2015/source/images/

mnistdigits.jpg  scurve.png  waves.png


## Predicting

Once we have a satisfactory model (as determined by the validation or crossvalidation
metrics), use the `h2o.predict()` command to compute and store
predictions on new data for additional refinements in the interactive data science
process.

In [17]:
predictions = model.predict(test_df)

deeplearning prediction progress: |███████████████████████████████████████████████████████████| 100%


## Variable Importance

Variable importance allows us to view the absolute and relative predictive strength of
each feature in the prediction task.
Each H2O algorithm class has its own methodology for computing variable importance.

You can enable the variable importance, by setting the `variable_importances` parameter to `True`.

H2O’s Deep Learning uses the Gedeon method [Gedeon, 1997](http://users.cecs.anu.edu.au/~Tom.Gedeon/pdfs/ContribDataMinv2.pdf), which is disabled
by default since it can be slow for large networks. 

If variable importance is a top priority in your analysis, consider training a Distributed Random Forest (DRF) model and compare the generated variable importances.

In [18]:
# Train Deep Learning model and validate on test set and save the variable importances
model_variable_importances = H2ODeepLearningEstimator(
     distribution="multinomial",
     activation="RectifierWithDropout",
     hidden=[32,32,32],
     input_dropout_ratio=0.2,
     sparse=True,
     l1=1e-5,
     epochs=10,
     variable_importances=True)

In [19]:
model_variable_importances.train(
         x=x,
         y=y,
         training_frame=train_df,
         validation_frame=test_df)

deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


In [20]:
import pandas as pd

In [21]:
# Retrieve the variable importance
pd.DataFrame(model_variable_importances.varimp())

,0,1,2,3
0,C351,1.000000,1.000000,0.003034
1,C434,0.896221,0.896221,0.002719
2,C516,0.873754,0.873754,0.002651
3,C379,0.864693,0.864693,0.002624
4,C377,0.836204,0.836204,0.002537
5,C489,0.828099,0.828099,0.002513
6,C323,0.826942,0.826942,0.002509
7,C488,0.823624,0.823624,0.002499
8,C378,0.813375,0.813375,0.002468
9,C517,0.804127,0.804127,0.002440


## Model Comparison with Grid Search

Grid search provides more subtle insights into the model tuning and selection
process by inspecting and comparing our trained models after the grid search process is complete. 

To learn when and how to select different parameter
configurations in a grid search, refer to Parameters for parameter descriptions
and configurable values.

There are different strategies to explore the hyperparameter combinatorial space:

- Cartesian Search: test *every* single combination
- Random Search: sample combinations

## Cartesian Search
In this example, three different network topologies and two different $l_1$ norm
weights are specified. This grid search model trains six different models using all
possible combinations of these parameters; other parameter combinations can
be specified for a larger space of models. Note that the models will most likely
converge before the default value of epochs, since early stopping is enabled.

In [22]:
from h2o.grid.grid_search import H2OGridSearch

In [23]:
hyper_parameters = {
    "hidden":[[32,32],[32,16,8],[100]], 
    "l1":[1e-4,1e-3],
}

model_grid = H2OGridSearch(H2ODeepLearningEstimator, hyper_params=hyper_parameters)

In [24]:
model_grid.train(
    x=x, 
    y=y,
    distribution="multinomial", 
    epochs=1000,
    training_frame=train_df, 
    validation_frame=test_df,
    score_interval=2, 
    stopping_rounds=3,
    stopping_tolerance=0.05,
    stopping_metric="misclassification")

deeplearning Grid Build progress: |███████████████████████████████████████████████████████████| 100%


In [25]:
# print model grid search results
model_grid

          hidden      l1  \
0    [I@1630e297  1.0E-4   
1    [I@3f11780b   0.001   
2    [I@7c471066   0.001   
3    [I@5243e69e   0.001   
4    [I@4376821b  1.0E-4   
5    [I@77136b5e  1.0E-4   

                                                              model_ids  \
0  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_2   
1  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_3   
2  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_4   
3  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_5   
4  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_0   
5  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_1   

               logloss  
0  0.09704577558818726  
1   0.1171770887627187  
2  0.11817273525829779  
3  0.12166162883662959  
4  0.14236506996926554  
5  0.16071863732335312  


In [26]:
for model in model_grid:
    print model.model_id + " mse: " + str(model.mse())

Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_2 mse: 0.00974703867597
Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_3 mse: 0.0255292775717
Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_4 mse: 0.0218750510677
Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_5 mse: 0.028621405161
Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_0 mse: 0.0103922185059
Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_1 mse: 0.0124590008794


In [27]:
import pandas as pd

In [28]:
grid_results = pd.DataFrame([[m.model_id, m.mse] for m in model_grid])

## Random Grid Search

If the search space is too large you can let the GridSearch algorithm select the parameter, by sampling from the parameter space. 

Just specify how many models (and/or how much training time) you want, and provide a seed to make the random selection deterministic.

In [29]:
hyper_parameters = {
    "hidden":[[17,32],[8,19],[32,16,8],[100],[10,10,10,10]], 
    "l1":[s/1e6 for s in range(1,1001)]
}

In [30]:
search_criteria = {"strategy":"RandomDiscrete",
"max_models":10, "max_runtime_secs":100,
 "seed":123456}

model_grid_random_search = H2OGridSearch(H2ODeepLearningEstimator,
    hyper_params=hyper_parameters,
    search_criteria=search_criteria)

In [31]:
model_grid_random_search.train(x=x, y=y,
                distribution="multinomial", 
                epochs=1000,
                training_frame=train_df, 
                validation_frame=test_df,
                score_interval=2, 
                stopping_rounds=3,
                stopping_tolerance=0.05,
                stopping_metric="misclassification")

deeplearning Grid Build progress: |███████████████████████████████████████████████████████████| 100%


In [32]:
grid_results = pd.DataFrame([[m.model_id, m.mse] for m in model_grid_random_search])

In [33]:
grid_results

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_5_model_1
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 72,810 weights/biases, 960.6 KB, 799,807 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,0.0,,,,,,,,,
,2,100,Rectifier,0.0,0.000355,0.0,0.2116555,0.3581470,0.0,0.0007051,0.0244147,0.0624240,0.1542075
,3,10,Softmax,,0.000355,0.0,0.1065168,0.2435885,0.0,-0.1683261,0.3856277,-0.0193657,0.0288906




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0191596950211
RMSE: 0.138418550134
LogLoss: 0.0741002274902
Mean Per-Class Error: 0.021834514392
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
1029.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0067568,"7 / 1,036"
0.0,1085.0,9.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0136364,"15 / 1,100"
5.0,2.0,973.0,2.0,1.0,0.0,2.0,5.0,2.0,1.0,0.0201410,20 / 993
5.0,0.0,7.0,945.0,0.0,3.0,1.0,6.0,5.0,2.0,0.0297741,29 / 974
2.0,2.0,0.0,0.0,958.0,0.0,0.0,2.0,1.0,7.0,0.0144033,14 / 972
4.0,1.0,2.0,10.0,1.0,860.0,4.0,0.0,1.0,0.0,0.0260476,23 / 883
6.0,1.0,0.0,0.0,1.0,3.0,962.0,1.0,3.0,0.0,0.0153531,15 / 977
0.0,0.0,8.0,2.0,0.0,1.0,0.0,987.0,1.0,4.0,0.0159521,"16 / 1,003"
6.0,5.0,0.0,3.0,0.0,4.0,2.0,2.0,913.0,8.0,0.0318134,30 / 943
2.0,4.0,0.0,6.0,11.0,6.0,0.0,12.0,2.0,924.0,0.0444674,43 / 967


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.9656
2,0.9874
3,0.9936
4,0.9962
5,0.9976
6,0.9986
7,0.9992
8,0.9997
9,0.9999
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-11 10:34:36,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-11 10:34:39,6.139 sec,37235 obs/sec,1.6662667,1,99976.0,0.2224147,0.1967078,0.0528661,0.2307572,0.2156277,0.0582
,2016-10-11 10:34:48,14.481 sec,38069 obs/sec,6.6691167,4,400147.0,0.1732573,0.1176858,0.0340288,0.1936166,0.1517399,0.0436
,2016-10-11 10:34:53,20.493 sec,37882 obs/sec,10.00035,6,600021.0,0.1604078,0.1000170,0.0295726,0.1878244,0.1420441,0.0412
,2016-10-11 10:35:01,27.871 sec,39818 obs/sec,15.0050167,9,900301.0,0.1533772,0.0908224,0.0271420,0.1806715,0.1374656,0.0386
,2016-10-11 10:35:09,35.396 sec,40482 obs/sec,19.9978167,12,1199869.0,0.1449157,0.0831624,0.0241037,0.1727691,0.1239685,0.0344
,2016-10-11 10:35:14,40.669 sec,40735 obs/sec,23.3303167,14,1399819.0,0.1482756,0.0858864,0.0255216,0.1780951,0.1309292,0.037
,2016-10-11 10:35:21,47.664 sec,39181 obs/sec,26.6634833,16,1599809.0,0.1411046,0.0791900,0.0222807,0.1779521,0.1301909,0.0374
,2016-10-11 10:35:28,55.190 sec,39690 obs/sec,31.6637667,19,1899826.0,0.1376266,0.0722149,0.0221795,0.1744335,0.1250483,0.0358
,2016-10-11 10:35:29,55.704 sec,39683 obs/sec,31.6637667,19,1899826.0,0.1449157,0.0831624,0.0241037,0.1727691,0.1239685,0.0344


,0,1
0,Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_5_model_1,<bound method H2ODeepLearningEstimator.mse of >
1,Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_5_model_0,<bound method H2ODeepLearningEstimator.mse of >


## Model Checkpoints 



H2O supporst model checkpoints. You can store the `state` of training and resume it later.
Checkpointing can be used to reload existing models that were saved to
disk in a previous session. 

To resume model training, use checkpoint model keys (model id) to incrementally
train a specific model using more iterations, more data, different data, and
so forth. To further train the initial model, use it (or its key) as a checkpoint
argument for a new model.

To improve this initial model, start from the previous model and add iterations by
building another model, specifying checkpoint=previous model id, and
changing train samples per iteration, target ratio comm to comp,
or other parameters. Many parameters can be changed between checkpoints,
especially those that affect regularization or performance tuning.

You can use GridSearch with checkpoint restarts to scan a broader range of hyperparameter combinations.

In [34]:
# Re-start the training process on a saved DL model using the ‘checkpoint‘ argument
model_checkpoint = H2ODeepLearningEstimator(
     checkpoint=model,
     distribution="multinomial",
     activation="RectifierWithDropout",
     hidden=[32,32,32],
     input_dropout_ratio=0.2,
     sparse=True,
     l1=1e-5,
     epochs=20)

In [35]:
model_checkpoint.train(
 x=x,
 y=y,
 training_frame=train_df,
 validation_frame=test_df)

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_1
Status of Neuron Layers: predicting C785, 10-class classification, multinomial distribution, CrossEntropy loss, 23,730 weights/biases, 386.1 KB, 2,800,033 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,717,Input,0.0,,,,,,,,,
,2,32,Rectifier,0.0,0.0001,0.0,0.0381236,0.0954844,0.0,0.0024246,0.0612324,0.1541252,0.3244088
,3,16,Rectifier,0.0,0.0001,0.0,0.0004519,0.0003064,0.0,-0.0259404,0.2158539,0.9568885,0.1034239
,4,8,Rectifier,0.0,0.0001,0.0,0.0004672,0.0003693,0.0,0.0519513,0.3624040,1.0016325,0.0751757
,5,10,Softmax,,0.0001,0.0,0.0237173,0.1108748,0.0,-0.9843064,1.7043371,-0.4039703,2.4672832




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0124590008794
RMSE: 0.11161989464
LogLoss: 0.0509048654871
Mean Per-Class Error: 0.0152310197673
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
1002.0,0.0,5.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0098814,"10 / 1,012"
1.0,1102.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0045167,"5 / 1,107"
1.0,1.0,982.0,3.0,1.0,1.0,1.0,5.0,2.0,0.0,0.0150451,15 / 997
0.0,1.0,4.0,1038.0,0.0,4.0,0.0,0.0,5.0,2.0,0.0151803,"16 / 1,054"
0.0,1.0,1.0,0.0,929.0,1.0,3.0,1.0,0.0,9.0,0.0169312,16 / 945
0.0,0.0,0.0,7.0,0.0,865.0,5.0,0.0,2.0,2.0,0.0181612,16 / 881
2.0,2.0,1.0,0.0,0.0,1.0,1002.0,0.0,0.0,0.0,0.0059524,"6 / 1,008"
2.0,1.0,3.0,1.0,3.0,0.0,1.0,1015.0,2.0,5.0,0.0174250,"18 / 1,033"
2.0,1.0,2.0,3.0,2.0,6.0,2.0,0.0,938.0,2.0,0.0208768,20 / 958
0.0,0.0,3.0,6.0,10.0,1.0,0.0,5.0,3.0,960.0,0.0283401,28 / 988


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.9636
2,0.9851
3,0.9924
4,0.9963
5,0.9971
6,0.9989
7,0.9993
8,0.9997
9,0.9999
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-11 10:27:42,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-11 10:27:45,51.125 sec,41072 obs/sec,1.6682167,1,100093.0,0.2181967,0.1943096,0.0514875,0.2331739,0.2284929,0.0587
,2016-10-11 10:27:52,58.725 sec,41841 obs/sec,6.6654167,4,399925.0,0.1520775,0.0933685,0.0268456,0.1942291,0.1667775,0.0426
,2016-10-11 10:27:58,1 min 3.921 sec,42021 obs/sec,10.0017667,6,600106.0,0.1423809,0.0843628,0.0232395,0.1904264,0.1674386,0.0407
,2016-10-11 10:28:03,1 min 9.088 sec,42253 obs/sec,13.33815,8,800289.0,0.1374043,0.0786025,0.0223380,0.1909020,0.1769573,0.0425
,2016-10-11 10:28:08,1 min 14.740 sec,42288 obs/sec,16.6745167,10,1000471.0,0.1286567,0.0660143,0.0175298,0.1855366,0.1621595,0.0395
,2016-10-11 10:28:19,1 min 25.153 sec,44749 obs/sec,25.0143667,15,1500862.0,0.1133383,0.0515222,0.0148252,0.1789131,0.1650323,0.0366
,2016-10-11 10:28:25,1 min 31.628 sec,45605 obs/sec,30.0060833,18,1800365.0,0.1116199,0.0509049,0.0150255,0.1775863,0.1607186,0.0364
,2016-10-11 10:28:32,1 min 37.905 sec,46352 obs/sec,35.0007,21,2100042.0,0.1127696,0.0520168,0.0151257,0.1810088,0.1723839,0.0376
,2016-10-11 10:28:38,1 min 44.497 sec,46901 obs/sec,40.0007,24,2400042.0,0.1082968,0.0464820,0.0138235,0.1810923,0.1712807,0.0376


deeplearning Model Build progress: |██████████████████████████████████████████████████████████| 100%


Specify a model and a file path. The default path is the current working directory.

In [36]:
model_path = h2o.save_model(
     model = model,
     #path = "/tmp/mymodel",
     force = True)

print model_path

/home/fmilo/workspace/h2o-deepwater/h2o-py/demos/Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_1


In [37]:
!ls -lah $model_path

-rw-rw-r-- 1 fmilo fmilo 442K Oct 11 10:36 /home/fmilo/workspace/h2o-deepwater/h2o-py/demos/Grid_DeepLearning_py_1_sid_9c08_model_python_1476206324318_4_model_1


After restarting H2O, you can load the saved model by specifying the host and model file path. 

Note: The saved model must be the same version used to save the model.

In [38]:
# Load model from disk
saved_model = h2o.load_model(model_path)

You can also use the following commands to retrieve a model from its H2O key.
This is useful if you have created an H2O model using the web interface and
want to continue the modeling process in another language, for example **R**.

In [39]:
# Retrieve model by H2O key
model = h2o.get_model(model_id=model_checkpoint._id)

## Conclusions

In this jupyter you learned to:
- use a deeplearning model
- use GridSearch
- use Checkpointing
- use Early Stopping